In [ ]:
%pip install ipywidgets
%pip install matplotlib
%pip install numpy

In [4]:

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})

width= 5
aspect_ratio= 1.13
fig_size = (width, width / aspect_ratio)
plt.figure(figsize=fig_size, constrained_layout=True)

# interact
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual

<Figure size 360x318.584 with 0 Axes>

In [5]:
@interact(dR = (40, 150, 5), Eb = (3000, 40000, 100))
def show(dR = 40, Eb = 3250):
    eV = 1.0 / 27.2114
    cmn1 = 1.0/219474.63

    # particle in a box potential
    def V1D(R, dR= dR, Eb = Eb):
        
        Eb = Eb * cmn1
        
        V  =  (R > dR/2) * Eb + (R < -dR/2) * Eb
        return V


    # Kinetic energy for electron 
    def Te(re):
        N = float(len(re))
        mass = 1.0
        Tij = np.zeros((int(N),int(N)))
        Rmin = float(re[0])
        Rmax = float(re[-1])
        step = float((Rmax-Rmin)/N)
        K = np.pi/step

        for ri in range(int(N)):
            for rj in range(int(N)):
                if ri == rj:  
                    Tij[ri,ri] = (0.5/mass)*K**2.0/3.0*(1+(2.0/N**2)) 
                else:    
                    Tij[ri,rj] = (0.5/mass)*(2*K**2.0/(N**2.0))*((-1)**(rj-ri)/(np.sin(np.pi*(rj-ri)/N)**2)) 
        return Tij

 
    Eb  = Eb
    nR  = 128
    Rmax = 100
    R = np.linspace(-Rmax,Rmax, nR)
    V = V1D(R,dR,Eb) 

    plt.plot(R,  V/cmn1, c =  "#000000", lw = 3)
    plt.ylim(-10, 7000)


    Hij = Te(R) + np.diag(V)
    Ei, Um   = np.linalg.eigh(Hij)
    color = ['#ef5777', '#48dbfb', '#8854d0', '#079992', '#e58e26',"#ff9ff3"]


    for i in range(0,3):
        plt.plot(R, Um[:,i]*0 + Ei[i]/cmn1, c =  "#000000")
        arg = np.argmax(np.abs(Um[:len(Um[:,i])//2,i]))
        sgn = Um[arg,i]/np.abs(Um[arg,i])
        plt.plot(R,sgn * Um[:,i] * 4000 + Ei[i]/cmn1, c =  color[i], ls = '-')
        plt.fill_between(R,Ei[i]/cmn1,  sgn* Um[:,i] * 4000 + Ei[i]/cmn1, facecolor=color[i], alpha = 0.3)

    plt.xlim(-100, 100)

    
    plt.xlabel("R (a.u.)")
    plt.ylabel("Energy (cm$^{-1}$)")
 

interactive(children=(IntSlider(value=40, description='dR', max=150, min=40, step=5), IntSlider(value=3250, de…